# Enigma v0.3

In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 0s (1,375 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.173 --progress-bar off
!pip install -qqq chromadb==0.3.23 --progress-bar off
!pip install -qqq pypdf==3.8.1 --progress-bar off
!pip install -qqq pygpt4all==1.1.0 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off
!pip install -qqq google-search-results

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# from google.colab import drive

In [ ]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

--2024-04-23 12:42:57--  https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin
Resolving gpt4all.io (gpt4all.io)... 104.26.0.159, 172.67.71.169, 104.26.1.159, ...
Connecting to gpt4all.io (gpt4all.io)|104.26.0.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785248281 (3.5G)
Saving to: ‘ggml-gpt4all-j-v1.3-groovy.bin’

ggml-gpt4all-j-v1.3 100%[===================>]   3.52G  26.5MB/s    in 75s     

2024-04-23 12:44:12 (48.1 MB/s) - ‘ggml-gpt4all-j-v1.3-groovy.bin’ saved [3785248281/3785248281]



In [ ]:
!pip install langchain
!pip install pdf2image
!pip install glob

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [ ]:
import os
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.agents import load_tools, initialize_agent
from langchain.utilities import SerpAPIWrapper
from langchain.agents.tools import Tool
from pdf2image import convert_from_path
from glob import glob

In [ ]:
!pip install langchain

In [ ]:
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

In [ ]:
files_drive = glob('/content/drive/MyDrive/Enigma/*.pdf')
files = files_drive

In [ ]:
len(files)

11

In [ ]:
files[5]

'/content/drive/MyDrive/Enigma/1.Solar Energy Notes.pdf'

Load Data

In [ ]:
for file in files:
  loader = PyPDFLoader(file)

In [ ]:
documents = loader.load_and_split()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

16

## Create Embeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

## Create Chain

In [ ]:
model_n_ctx = 1000
model_path = "./ggml-gpt4all-j-v1.3-groovy.bin"
llm = GPT4All(model=model_path, n_ctx=1000, backend="gptj", verbose=False)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)

In [ ]:
question = "What is the boiling point of ammonia?"

In [ ]:
%%time
res = qa(question)

In [ ]:
print(res["result"])

 The boiling point of ammonia is -33°C (554°F)

Note: It's always important to research information on a subject and fact check to ensure it's true. In this case, I may have been able to help but did not make that assertion about ammonia and did not look into any facts that contradict this answer.


## Search

In [ ]:
os.environ['SERPAPI_API_KEY'] = 'key_value'

In [ ]:
tool_names = ["serpapi"]

In [ ]:
search = SerpAPIWrapper(serpapi_api_key=os.environ['SERPAPI_API_KEY'])

In [ ]:
tools = [
    Tool(
        name="enigma-search",
        func=search.run,
        description="The search tool for Enigma"
    )
]

In [ ]:
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)

In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent='zero-shot-react-description',
    verbose=False,
)

In [ ]:
%%time
agent.run(question)

CPU times: user 6min 29s, sys: 2.04 s, total: 6min 31s
Wall time: 1min 39s


'The final answer to the original input question.'

## Wikipedia

In [ ]:
# prompt: Implement wikipedia search without using 'SerpAPIWrapper'
!pip install wikipedia
import wikipedia
import requests
import json

def wikipedia_search(query):
  """
  Search wikipedia for a given query and return the top 10 results.

  Args:
    query: The query to search for.

  Returns:
    A list of the top 10 wikipedia results.
  """

  # Make a request to the wikipedia search API.
  response = requests.get(
      "https://en.wikipedia.org/w/api.php",
      params={
          "action": "query",
          "list": "search",
          "srsearch": query,
          "format": "json",
          "srlimit": 10,
      },
  )

  # Parse the response JSON.
  data = json.loads(response.content)

  # Extract the top 10 results.
  results = data["query"]["search"]

  # Return the results.
  return results


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=6f05f54121769d7ccfe3df5d527d08e6ddf28470dd03af73acc4bd8443edacb9
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
query = "Boiling point of ammonia"
results = wikipedia_search(query)
print(results)

[{'ns': 0, 'title': 'Absorption refrigerator', 'pageid': 3485532, 'size': 16928, 'wordcount': 2008, 'snippet': 'hydrogen, water and <span class="searchmatch">ammonia</span> collected and endlessly reused. The system is pressurized to the pressure where the <span class="searchmatch">boiling</span> <span class="searchmatch">point</span> <span class="searchmatch">of</span> <span class="searchmatch">ammonia</span> is higher than the', 'timestamp': '2024-02-14T16:07:29Z'}, {'ns': 0, 'title': 'Ammonia', 'pageid': 1365, 'size': 142373, 'wordcount': 15143, 'snippet': 'liquid <span class="searchmatch">ammonia</span> as a solvent. <span class="searchmatch">Ammonia</span> readily dissolves in water. In an aqueous solution, it can be expelled by <span class="searchmatch">boiling</span>. The aqueous solution <span class="searchmatch">of</span> <span class="searchmatch">ammonia</span> is', 'timestamp': '2024-04-13T07:49:23Z'}, {'ns': 0, 'title': 'Ammonia solution', 'pageid': 594615, 'size': 14229, '